In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')

In [ ]:
!pip install textstat

In [ ]:
!mkdir pip; cd pip; pip download textstat pyphen

In [ ]:
import textstat

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X = df['excerpt'].values
Y = df['target'].values

In [ ]:
tfidf.fit(X, Y)

In [ ]:
# Voy a hacer el fit_transform directamente porque es lo que recomienda la doc: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# Haciéndolo así, el paso anterior (fit) me lo puedo saltar
Xtfidf = tfidf.fit_transform(X)
Xtfidf

In [ ]:
vector_primer_texto = Xtfidf[0]

In [ ]:
vector_primer_texto[vector_primer_texto != 0]

In [ ]:
vector_primer_texto[vector_primer_texto != 0].max()

In [ ]:
vector_primer_texto[vector_primer_texto != 0].min()

In [ ]:
reg = LinearRegression().fit(Xtfidf, Y)

In [ ]:
reg.coef_.shape

In [ ]:
pred = reg.predict(Xtfidf)

In [ ]:
# Muestra las predicciones
pred

In [ ]:
# Muestra los targets
Y

In [ ]:
# Un poco sospechoso... Vamos a ver el RMSE
mean_squared_error(pred, Y, squared=False)

# Hay sobreajuste porque tenemos nº de muestras << nº parámetros
https://witeboard.com/8f405e60-de37-11eb-8ca9-7b665841bc76

# Solucion al sobreajuste (overfit)
Vamos a hacer la misma regresión PERO validando con 5-folds. Esto **no** va a evitar el sobreajuste a los datos de entrenamiento, pero nos permitirá comprobar cómo generaliza el modelo prediciendo datos de validación (que no ha visto nunca).

Pero antes, un inciso para que tengamos claro el contenido de la matriz Xtfidf: Xtfidf es la matriz de índices TF-IDF por cada palabra de nuestro vocabulario (columnas) y por cada extracto (filas).

Con `Xtfidf.shape` podemos ver que es una matriz bastante grande (2834 x 26833). Como cada texto tiene entre 100 y 300 palabras aprox y el índice TF-IDF de una palabra que no aparece en el texto es 0, se trata de una matriz dispersa donde la gran mayoría de valores estarán a 0 y solo unos pocos tendrán valores distintos de 0.

In [ ]:
Xtfidf.shape

In [ ]:
type(Xtfidf)

## A partir de Xtfidf, ¿Cuántas palabras hay en cada texto?

In [ ]:
(Xtfidf != 0).sum(axis=1)

## ¿En cuántos documentos aparece cada palabra?

In [ ]:
(Xtfidf != 0).sum(axis=0)

# ¿Cuál es el porcentaje de datos distintos de 0 en la matrix Xtfidf?

In [ ]:
f'{(Xtfidf != 0).sum() / (Xtfidf.shape[0] * Xtfidf.shape[1]) * 100:.3f}%'

# Ahora sí: entrena con 5 folds

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.decomposition import PCA, TruncatedSVD
from tqdm.auto import tqdm

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    LinearRegression()
)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=33)

In [ ]:
for train_idx, valid_idx in tqdm(kf.split(X), total=5):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    print(mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False))

In [ ]:
mean_squared_error(df['pred'], df['target'], squared=False)

# ¿Funcionaría mejor con regularización?

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    Ridge()
)

In [ ]:
for train_idx, valid_idx in tqdm(kf.split(X), total=5):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    print(mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False))

In [ ]:
mean_squared_error(df['pred'], df['target'], squared=False)

In [ ]:
!mkdir model

In [ ]:
from joblib import dump, load

In [ ]:
# Comentamos esta línea porque más abajo los vamos a guardar todos
#dump(model, 'model/tfidf_ridge_fold_5_cv_0.7177.joblib')

# Ejercicios
- ¿Cuántos coeficientes tenemos en nuestro regresor Ridge? (la dificultad de este ejercicio es que ahora nuestro regresor está dentro de un `pipeline`. Deben ser tantos como palabras haya en nuestro vocabulario, o columnas tenga `Xtfidf`)
- ¿Cuántos de ellos son 0 gracias a la regularización L2? (cuanto más coeficientes a 0 haya, más efectiva habrá sido la regularización del modelo)
- Probar con regularización L1 (Lasso) y ver si mejora el CV
- Probar cambiando el hiperparámetro `alpha` de Ridge/Lasso
- Usar algún reductor de dimensionalidad (ej. PCA) para reducir la complejidad del modelo y ver si mejora. 
  - Puntos extra por integrar PCA dentro del `pipeline`.
- Muestra el vocabulario aprendido por el vectorizador TF-IDF
- ¿Qué palabra es la que más se repite en todo el corpus?

# Soluciones

* ¿Cuántos coeficientes tenemos en nuestro regresor Ridge? (la dificultad de este ejercicio es que ahora nuestro regresor está dentro de un pipeline. Deben ser tantos como palabras haya en nuestro vocabulario, o columnas tenga Xtfidf)

In [ ]:
model[1].coef_.shape

* ¿Cuántos de ellos son 0 gracias a la regularización L2? (cuanto más coeficientes a 0 haya, más efectiva habrá sido la regularización del modelo)

In [ ]:
(model[1].coef_ == 0).sum()

* Probar con regularización L1 (Lasso) y ver si mejora el CV

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    Lasso(alpha=1)
)

for train_idx, valid_idx in tqdm(kf.split(X), total=5):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    print(mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False))

* Probar cambiando el hiperparámetro alpha de Ridge/Lasso

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    Lasso(alpha=0.0001)
)

for train_idx, valid_idx in tqdm(kf.split(X), total=5):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    print(mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False))

* Usar algún reductor de dimensionalidad (ej. PCA) para reducir la complejidad del modelo y ver si mejora.

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    TruncatedSVD(100),
    Ridge()
)
for train_idx, valid_idx in tqdm(kf.split(X), total=5):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    print(mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False))
print("cv", mean_squared_error(df['pred'], df['target'], squared=False))

# Ejercicio
- Persistir los **5 modelos**. Los vamos a llamar `tfidf_ridge_fold_N_cv_X.XXXX.joblib`, donde N = número de fold (del 1 al 5) y X.XXXX es el RMSE del fold con 4 decimales. Ver las celdas de abajo para truco útil para nombrar el modelo.

In [ ]:
a = 0.1234567890

In [ ]:
print("La variable a vale", a)

In [ ]:
print(f"La variable a vale {a:.3f}")

In [ ]:
# Todo lo que hay entre las {} se evalúa. Ejemplo:
print(f"La variable a vale {a+1:.3f}")

# Solución

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    Ridge()
)

for fold, (train_idx, valid_idx) in enumerate(tqdm(kf.split(X), total=5)):
    Xt = X[train_idx]
    Yt = Y[train_idx]
    Xv = X[valid_idx]
    reg = model.fit(Xt, Yt)
    df.loc[valid_idx, 'pred'] = model.predict(Xv)
    val_rmse = mean_squared_error(df.loc[valid_idx, 'pred'], df.loc[valid_idx, 'target'], squared=False)
    file_name = f"model/tfidf_ridge_fold_{fold+1}_cv_{val_rmse:.4f}.joblib"
    print(file_name)
    dump(model, file_name)